In [1]:
import pandas as pd

In [2]:
df = pd.read_json('resources/datasets/val_sentence_pairs.json')
df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2


In [3]:
titan_swapped_df  = pd.read_json('results/Titan-model-output-swapped.json')
titan_no_swap_df  = pd.read_json('results/Titan-model-output-no-swap.json')

In [4]:
titan_swapped_df.head()

,output
0,B
1,B
10,B
100,B
101,B


In [5]:
titan_swapped_df.shape

(373, 1)

In [6]:
titan_swapped_df.rename(columns={'output': 'titan_swapped_output'}, inplace=True)
titan_no_swap_df.rename(columns={'output': 'titan_no_swap_output'}, inplace=True)

In [7]:
input_df = df

combined_df = pd.concat([input_df, 
                         titan_swapped_df, 
                         titan_no_swap_df,], axis=1)
combined_df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id,titan_swapped_output,titan_no_swap_output
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0,B,B
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1,B,B
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1,B,A
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2,B,A
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2,B,A


In [8]:
def is_correct(row): 
    if (row[0] == 'B' and row[1] == 'A'):
        return 'Y'
    if (row[0] == 'A' and row[1] == 'A'):
        return 'AA'
    if (row[0] == 'B' and row[1] == 'B'):
        return 'BB'
    if (row[0] == 'A' and row[1] == 'B'):
        return 'N'
    return 'No answer'

In [9]:
combined_df['titan_analysis_result'] = combined_df[['titan_swapped_output', 'titan_no_swap_output']].apply(is_correct, axis=1)

In [10]:
def acc_bias(col):
    results = combined_df[col].value_counts()/len(combined_df)
    acc = results['Y']
    in_acc = results['N']
    no_answer = results['No answer']
    if 'AA' not in results:
        results['AA'] = 0
    if 'BB' not in results: 
        results['BB'] = 0
        
    bias = abs(results['AA'] - results['BB'])
    aa_ratio = results['AA'] * 100
    bb_ratio = results['BB'] * 100
    if results['AA'] > results['BB']:
        towards = 'A'
    else:
        towards = 'B'
        
    return acc, in_acc, no_answer, aa_ratio, bb_ratio, bias, towards

In [12]:
for m in ['titan',]:
    acc, in_acc, no_answer, aa_ratio, bb_ratio, bias, towards = acc_bias(m+'_analysis_result')
    acc = acc*100
    in_acc = in_acc*100
    no_answer = no_answer*100
    bias = bias*100
    print(f'{m}:   \tAccuracy: {acc:.1f}%     \tInaccuracy: {in_acc:.1f}%     \tNO_Answer: {no_answer:.1f}%     \tAA: {aa_ratio:.1f}%\tBB: {bb_ratio:.1f}%\tBias: {bias:.1f}% towards {towards}')

titan:   	Accuracy: 65.4%     	Inaccuracy: 2.4%     	NO_Answer: 1.9%     	AA: 2.1%	BB: 28.2%	Bias: 26.0% towards B
